計算 Moving Average 策略究竟能否賺錢?


2021.09.11

## Strategy Evaluation

注意到這邊我們先忽略 spreads

In [2]:
import pandas as pd
import utils

import plotly.graph_objects as go 

In [5]:
import instrument # 引入 instrument class

In [3]:
pair = 'EUR_USD'
granularity = 'H1'
ma_list = [16, 64] # loop through this much MA

In [6]:
# 取的欲交易商品的資訊
# refer to instrument.py

i_pair = instrument.Instrument.get_instrument_by_name(pair)

In [7]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

non_cols  = ['time', 'volume']
mod_cols = [x for x in df.columns if x not in non_cols]
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

### 製作多條 MA

In [8]:
df_ma = df[ ['time', 'mid_o', 'mid_h', 'mid_l', 'mid_c'] ].copy()

for ma in ma_list:
    df_ma[f'MA_{ma}'] = df_ma['mid_c'].rolling(window=ma).mean()

df_ma.dropna(inplace=True)

### 佈署策略

In [9]:
df_ma['DIFF'] = df_ma.MA_16 - df_ma.MA_64

# DIFF_PREV = 昨日 DIFF
df_ma['DIFF_PREV'] = df_ma['DIFF'].shift(1)

def is_trade(row): # this function should be executed row by row
    if row.DIFF >= 0 and row.DIFF_PREV < 0: # 進場做多
        return 1
    if row.DIFF <= 0 and row.DIFF_PREV > 0: # 進場做空
        return -1 
    else:  # 否則就不進場
        return 0

### 取出交易機會

In [10]:
df_ma['IS_TRADE'] = df_ma.apply(is_trade, axis=1) # apply function row by 

# 注意: 這邊要改成過濾出 IS_TRADE 不等於 0 的所有 row，因為 +1 代表進場做多，-1 代表進場做空
# 而必須要有一次完整的進出場，才能計算損益
df_trades = df_ma[df_ma['IS_TRADE'] != 0].copy()

In [11]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE
121,2021-01-25T17:00:00.000000000Z,1.21368,1.21475,1.21362,1.21445,1.215794,1.215855,-0.000060,0.000096,-1
153,2021-01-27T01:00:00.000000000Z,1.21624,1.21672,1.21584,1.21610,1.215634,1.215497,0.000137,-0.000083,1
164,2021-01-27T12:00:00.000000000Z,1.21164,1.21198,1.21056,1.21079,1.214941,1.215047,-0.000106,0.000171,-1
216,2021-01-29T16:00:00.000000000Z,1.21492,1.21534,1.21298,1.21352,1.212039,1.211844,0.000195,-0.000022,1
236,2021-02-01T12:00:00.000000000Z,1.20818,1.20887,1.20722,1.20756,1.211494,1.211547,-0.000052,0.000272,-1


### 計算價差 (不考慮 spread) (= 出場點價格 - 進場點價格)

以上圖來說，第一次交易是 2021.01.25 17:00 UTC 進場做空 (因為 DIFF 為負，且 DIFF_PREV 為正，代表"死亡交叉")  
出場點即是第二 row，2021.01.27 01:00 UTC  
因此損益為 - (1.21610-1.21445) 

In [14]:
df_trades['DELTA'] = df_trades['mid_c'].diff()

In [15]:
df_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA
121,2021-01-25T17:00:00.000000000Z,1.21368,1.21475,1.21362,1.21445,1.215794,1.215855,-0.000060,0.000096,-1,NaN
153,2021-01-27T01:00:00.000000000Z,1.21624,1.21672,1.21584,1.21610,1.215634,1.215497,0.000137,-0.000083,1,0.00165
164,2021-01-27T12:00:00.000000000Z,1.21164,1.21198,1.21056,1.21079,1.214941,1.215047,-0.000106,0.000171,-1,-0.00531
216,2021-01-29T16:00:00.000000000Z,1.21492,1.21534,1.21298,1.21352,1.212039,1.211844,0.000195,-0.000022,1,0.00273
236,2021-02-01T12:00:00.000000000Z,1.20818,1.20887,1.20722,1.20756,1.211494,1.211547,-0.000052,0.000272,-1,-0.00596
...,...,...,...,...,...,...,...,...,...,...,...
3803,2021-08-27T02:00:00.000000000Z,1.17496,1.17576,1.17494,1.17526,1.175463,1.175465,-0.000002,0.000164,-1,0.00390
3807,2021-08-27T06:00:00.000000000Z,1.17617,1.17700,1.17588,1.17682,1.175588,1.175554,0.000033,-0.000045,1,0.00156
3809,2021-08-27T08:00:00.000000000Z,1.17614,1.17628,1.17505,1.17512,1.175572,1.175583,-0.000010,0.000004,-1,-0.00170
3811,2021-08-27T10:00:00.000000000Z,1.17581,1.17638,1.17556,1.17590,1.175645,1.175610,0.000035,-0.000002,1,0.00078


注意到，用 ```pandas``` 的 ```.diff()``` 可計算兩 row 數值之差，  
但真正的損益應該是第二 row 的 ```DELTA```  
-
以上圖來說，第一筆交易的進場點是 1.21445，出場點是 1.21610，  
且因為進場時 ```IS_TRADE``` 是 -1，代表這筆交易是做空，  
因此損益為 - (1.21610-1.21445) = -0.00165 (即第二 row 的 ```DELTA```)

In [24]:
# 因此我們將 DELTA   上      移一格!!! (不是下移)
df_trades['DELTA'] = df_trades['mid_c'].diff().shift(-1)

In [25]:
df_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA
121,2021-01-25T17:00:00.000000000Z,1.21368,1.21475,1.21362,1.21445,1.215794,1.215855,-0.000060,0.000096,-1,0.00165
153,2021-01-27T01:00:00.000000000Z,1.21624,1.21672,1.21584,1.21610,1.215634,1.215497,0.000137,-0.000083,1,-0.00531
164,2021-01-27T12:00:00.000000000Z,1.21164,1.21198,1.21056,1.21079,1.214941,1.215047,-0.000106,0.000171,-1,0.00273
216,2021-01-29T16:00:00.000000000Z,1.21492,1.21534,1.21298,1.21352,1.212039,1.211844,0.000195,-0.000022,1,-0.00596
236,2021-02-01T12:00:00.000000000Z,1.20818,1.20887,1.20722,1.20756,1.211494,1.211547,-0.000052,0.000272,-1,-0.00265
...,...,...,...,...,...,...,...,...,...,...,...
3803,2021-08-27T02:00:00.000000000Z,1.17496,1.17576,1.17494,1.17526,1.175463,1.175465,-0.000002,0.000164,-1,0.00156
3807,2021-08-27T06:00:00.000000000Z,1.17617,1.17700,1.17588,1.17682,1.175588,1.175554,0.000033,-0.000045,1,-0.00170
3809,2021-08-27T08:00:00.000000000Z,1.17614,1.17628,1.17505,1.17512,1.175572,1.175583,-0.000010,0.000004,-1,0.00078
3811,2021-08-27T10:00:00.000000000Z,1.17581,1.17638,1.17556,1.17590,1.175645,1.175610,0.000035,-0.000002,1,0.01068


### 將價差單位轉為 pip

In [17]:
type(i_pair)

instrument.Instrument

In [18]:
i_pair.pipLocation

0.0001

In [37]:
df_trades['DELTA'] = ( df_trades['mid_c'].diff() / i_pair.pipLocation ).shift(-1)

In [38]:
df_trades

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
121,2021-01-25T17:00:00.000000000Z,1.21368,1.21475,1.21362,1.21445,1.215794,1.215855,-0.000060,0.000096,-1,16.5,-0.00165
153,2021-01-27T01:00:00.000000000Z,1.21624,1.21672,1.21584,1.21610,1.215634,1.215497,0.000137,-0.000083,1,-53.1,-0.00531
164,2021-01-27T12:00:00.000000000Z,1.21164,1.21198,1.21056,1.21079,1.214941,1.215047,-0.000106,0.000171,-1,27.3,-0.00273
216,2021-01-29T16:00:00.000000000Z,1.21492,1.21534,1.21298,1.21352,1.212039,1.211844,0.000195,-0.000022,1,-59.6,-0.00596
236,2021-02-01T12:00:00.000000000Z,1.20818,1.20887,1.20722,1.20756,1.211494,1.211547,-0.000052,0.000272,-1,-26.5,0.00265
...,...,...,...,...,...,...,...,...,...,...,...,...
3803,2021-08-27T02:00:00.000000000Z,1.17496,1.17576,1.17494,1.17526,1.175463,1.175465,-0.000002,0.000164,-1,15.6,-0.00156
3807,2021-08-27T06:00:00.000000000Z,1.17617,1.17700,1.17588,1.17682,1.175588,1.175554,0.000033,-0.000045,1,-17.0,-0.00170
3809,2021-08-27T08:00:00.000000000Z,1.17614,1.17628,1.17505,1.17512,1.175572,1.175583,-0.000010,0.000004,-1,7.8,-0.00078
3811,2021-08-27T10:00:00.000000000Z,1.17581,1.17638,1.17556,1.17590,1.175645,1.175610,0.000035,-0.000002,1,106.8,0.01068


### 計算損益 (= 價差 * 交易方向)

In [39]:
df_trades['GAIN'] = df_trades['DELTA'] * df_trades['IS_TRADE']

In [40]:
df_trades 
# 因此只要 DELTA 與 GAIN 完全相同，則代表該筆交易 MA 策略成功

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
121,2021-01-25T17:00:00.000000000Z,1.21368,1.21475,1.21362,1.21445,1.215794,1.215855,-0.000060,0.000096,-1,16.5,-16.5
153,2021-01-27T01:00:00.000000000Z,1.21624,1.21672,1.21584,1.21610,1.215634,1.215497,0.000137,-0.000083,1,-53.1,-53.1
164,2021-01-27T12:00:00.000000000Z,1.21164,1.21198,1.21056,1.21079,1.214941,1.215047,-0.000106,0.000171,-1,27.3,-27.3
216,2021-01-29T16:00:00.000000000Z,1.21492,1.21534,1.21298,1.21352,1.212039,1.211844,0.000195,-0.000022,1,-59.6,-59.6
236,2021-02-01T12:00:00.000000000Z,1.20818,1.20887,1.20722,1.20756,1.211494,1.211547,-0.000052,0.000272,-1,-26.5,26.5
...,...,...,...,...,...,...,...,...,...,...,...,...
3803,2021-08-27T02:00:00.000000000Z,1.17496,1.17576,1.17494,1.17526,1.175463,1.175465,-0.000002,0.000164,-1,15.6,-15.6
3807,2021-08-27T06:00:00.000000000Z,1.17617,1.17700,1.17588,1.17682,1.175588,1.175554,0.000033,-0.000045,1,-17.0,-17.0
3809,2021-08-27T08:00:00.000000000Z,1.17614,1.17628,1.17505,1.17512,1.175572,1.175583,-0.000010,0.000004,-1,7.8,-7.8
3811,2021-08-27T10:00:00.000000000Z,1.17581,1.17638,1.17556,1.17590,1.175645,1.175610,0.000035,-0.000002,1,106.8,106.8


In [41]:
df_trades.shape

(85, 12)

### 回測整個時間段

In [42]:
# 回測時間段為:
print(df['time'].iloc[0])
df['time'].iloc[-1]

2021-01-18T16:00:00.000000000Z


'2021-09-08T05:00:00.000000000Z'

In [43]:
df_trades['GAIN'].sum()
# 使用 EURUSD 1 小時圖，在近 8 個月使用 MA 策略，淨損益為 -269 pips

-269.70000000001284

### 思考為何 MA 策略聽起還很棒，但實際上 (以此例來說)賺不到錢

In [44]:
df_trades.head()

,time,mid_o,mid_h,mid_l,mid_c,MA_16,MA_64,DIFF,DIFF_PREV,IS_TRADE,DELTA,GAIN
121,2021-01-25T17:00:00.000000000Z,1.21368,1.21475,1.21362,1.21445,1.215794,1.215855,-0.000060,0.000096,-1,16.5,-16.5
153,2021-01-27T01:00:00.000000000Z,1.21624,1.21672,1.21584,1.21610,1.215634,1.215497,0.000137,-0.000083,1,-53.1,-53.1
164,2021-01-27T12:00:00.000000000Z,1.21164,1.21198,1.21056,1.21079,1.214941,1.215047,-0.000106,0.000171,-1,27.3,-27.3
216,2021-01-29T16:00:00.000000000Z,1.21492,1.21534,1.21298,1.21352,1.212039,1.211844,0.000195,-0.000022,1,-59.6,-59.6
236,2021-02-01T12:00:00.000000000Z,1.20818,1.20887,1.20722,1.20756,1.211494,1.211547,-0.000052,0.000272,-1,-26.5,26.5


In [45]:
df_plot = df_ma.iloc[115:255].copy() # 畫出前幾筆交易的蠟燭圖

In [46]:
fig = go.Figure() 

# candle stick:
fig.add_trace(go.Candlestick(
    x = df_plot.time,
    open = df_plot.mid_o,
    high = df_plot.mid_h,
    low = df_plot.mid_l,
    close = df_plot.mid_c,

    line=dict(width=1),
    opacity=1,

    increasing_fillcolor='#24A06B',
    decreasing_fillcolor="#CC2E3C",
    increasing_line_color='#2EC886',  
    decreasing_line_color='#FF3A4C'
))


# MA:
for ma in ma_list:
    col = f'MA_{ma}'

    fig.add_trace(go.Scatter(x=df_plot.time, 
    y=df_plot[col],
    line=dict(width=2),
    line_shape='spline', 
    name=f'MA_{ma}'
    ))


fig.update_layout(width=1000, height=400, 
    margin=dict(l=10, r=10, b=10, t=10), # 讓線圖更明顯
    font=dict(size=10, color='#e1e1e1'), # 將 x 軸資料轉為白色
    paper_bgcolor='#1e1e1e', # 將背景改為黑色
    plot_bgcolor='#1e1e1e') 

fig.update_xaxes( # 更新 x 軸資料
    gridcolor='#1f292f',
    showgrid=True, 
    fixedrange=True,
    rangeslider=dict(visible=False)
)

fig.update_yaxes( # 更新 y 軸資料
    gridcolor='#1f292f',
    showgrid=True, 
    fixedrange=True
)

用數據說話，不要用肉眼看!!!